# SI 650 / EECS 549: Homework 3 Part 3

Homework 3 Part 3 will have you working with deep learning models in a variety of ways. You will likely need to run this on Great Lakes unless you have access to a GPU elsewhere (or be prepared to wait a long time). You should have completed Parts 1 and 2 before attempting this notebook to familiarize yourself with how PyTerrier works.

In Part 3, you'll try the following tasks:
 - Use a large language model to re-rank content
 - Use a text-to-text model to perform query augmentation
 - Train a deep learning IR model and compare its performance.
 
The first two of these tasks will rely on models that we've pretrained for you. However, we've also provided code for how to train these. In the third task, you'll do one simple training and evaluate its results.

For the first two tasks, we've provided most of the code. *You are expected to submit results showing that you have successfully executed it*. You'll need to understand some of the code to complete task 3, which requires you to write new code.

As with the past notebooks, you'll need to have `JAVA_HOME` set, which will need to be run on Great Lakes. You can potentially set it in the notebook with a Jupyter command like
```
!export JAVA_HOME=/fill/in/the/path/to/the/JDK/here
```
by first figuring out where the JDK is installed. (This is setting the `JAVA_HOME` environment variable in the unix way). 

### Install the PyTerrier extensions

You'll need two extensions for [OpenNIR](https://opennir.net/) and [doc2query](https://github.com/terrierteam/pyterrier_doc2query). We've provided the package install commands in comments below.

In [1]:
#pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
#pip install --upgrade git+https://github.com/terrierteam/pyterrier_doc2query.git

## Getting started

Start PyTerrier as we have in past notebooks.

In [2]:
import pyterrier as pt
if not pt.started():
    pt.init(tqdm='notebook')
import onir_pt
import pyterrier_doc2query
import os
import pandas as pd

PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### [TREC-COVID19](https://ir.nist.gov/covidSubmit/) Dataset download

The following cell downloads the [TREC-COVID19](https://ir.nist.gov/covidSubmit/) dataset that we will use periodically throughout this notebook.

In [3]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

/home/cyuntao/miniconda3/envs/650/lib/python3.9/site-packages/pyterrier/datasets.py:430: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


# Task 1: Build the inverted index for the TREC-COVID19 collection. (2 points)

Build the index for the TREC Covid-19 (CORD19) data like we have in past notebooks but without any fancy options (e.g., no positional indexing).

In [4]:
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_trec_covid'

if not os.path.exists(pt_index_path + "/data.properties"):

    # create the index, using the IterDictIndexer indexer 

    # TODO
    indexer = pt.IterDictIndexer(pt_index_path)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    
    # TODO
    index_ref = indexer.index(cord19.get_corpus_iter(), fields=('abstract',), meta=('docno',))
else:
    # if you already have the index, create an IndexRef from the data in pt_index_path
    # that we can use to load using the IndexFactory
    
    # TODO
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
    
index = pt.IndexFactory.of(index_ref)

## Using an untuned Re-rankers

This notebook will have you work with a few neural re-ranking methods that you've used in class. We can build them from scratch using `onir_pt.reranker` or load them from pretrained models. The models we load from scratch won't have been trained to do IR (yet), however.

And OpenNIR reranking model consists of:
 - `ranker` (e.g., `drmm`, `knrm`, or `pacrr`). This defines the neural ranking architecture. We discussed the `knrm` approach in class.
 - `vocab` (e.g., `wordvec_hash`, or `bert`). This defines how text is encoded by the model. This approach makes it easy to swap out different text representations. 
 
Let's start with the `knrm` method we discussed in class:

In [5]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

config file not found: config
[2021-11-20 23:53:36,417][WordvecHashVocab][DEBUG] [starting] reading cached at /home/cyuntao/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2021-11-20 23:54:17,909][WordvecHashVocab][DEBUG] [finished] reading cached at /home/cyuntao/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [41.51s]


Let's look at how well this model work at ranking compared with our default `BatchRetrieve`

In [6]:
br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-20 23:54:39,697][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-20 23:55:18,840][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-20 23:55:21,081][onir_pt][DEBUG] [finished] batches: [2.24s] [1250it] [558.22it/s]


,name,map,ndcg,ndcg_cut.10,P.10,mrt
0,DPH,0.068056,0.165653,0.609058,0.658,71.727372
1,DPH >> KNRM,0.054808,0.145462,0.359895,0.450,1081.976549


The `knrm` models' performance is lower! The mode doesn't work very well because it hasn't yet been trained for IR; it's using random weights to combine the scores from the similarity matrix--but this is at least a start.

## Loading a trained re-ranker

You can train re-ranking models in PyTerrier using the `fit` method. Here's an example of how to train the `knrm` model on the MS MARCO dataset, which is a large IR collection.

```python
# transfer training signals from a medical sample of MS MARCO
from sklearn.model_selection import train_test_split
train_ds = pt.datasets.get_dataset('irds:msmarco-passage/train/medical')
train_topics, valid_topics = train_test_split(train_ds.get_topics(), test_size=50, random_state=42) # split into training and validation sets

# Index MS MARCO
indexer = pt.index.IterDictIndexer('./terrier_msmarco-passage')
tr_index_ref = indexer.index(train_ds.get_corpus_iter(), fields=('text',), meta=('docno',))

pipeline = (pt.BatchRetrieve(tr_index_ref) % 100 # get top 100 results
            >> pt.text.get_text(train_ds, 'text') # fetch the document text
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'})) # rename columns
            >> knrm) # apply neural re-ranker

pipeline.fit(
    train_topics,
    train_ds.get_qrels(),
    valid_topics,
    train_ds.get_qrels())
```

Training deep learning models takes a bit of time (especially for large datasets like MS MARCO), so we've provided a model that's already been trained for you to download.

In [7]:
del knrm # free up the memory before loading a new version of the ranker (helpful for the GPU)
knrm = onir_pt.reranker.from_checkpoint('http://jurgens.people.si.umich.edu/ir/knrm.medmarco.tar.gz', text_field='abstract', 
                                        expected_md5="d70b1d4f899690dae51161537e69ed5a")

[2021-11-20 23:55:21,238][onir_pt][INFO] using cached checkpoint: /home/cyuntao/data/onir/model_checkpoints/e40f18b6f6e8851dd92f4d1196a43875
[2021-11-20 23:55:21,241][WordvecHashVocab][DEBUG] [starting] reading cached at /home/cyuntao/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2021-11-20 23:55:22,715][WordvecHashVocab][DEBUG] [finished] reading cached at /home/cyuntao/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [1.47s]


In [8]:
pipeline2 = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline2],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-20 23:55:32,379][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-20 23:55:32,691][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-20 23:55:34,817][onir_pt][DEBUG] [finished] batches: [2.12s] [1250it] [588.25it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,46.558018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> KNRM,0.065109,0.598,0.160570,0.532790,91.907720,20.0,30.0,0.096531,12.0,26.0,0.024604,20.0,30.0,0.028582,20.0,30.0,0.006073


The tuned performance is a little better than before, but `knrm` still underperforms our first-stage ranking model.

## Reranking with BERT

Large language models such as [BERT](https://arxiv.org/abs/1810.04805) are much more powerful neural models that have been shown to be effective for ranking like we discussed in class. 

Like with `knrm`, we'll start by using BERT for re-ranking with its initial parameters. These parameters have been turned for the masked language modeling (i.e., filling a word in the blank) and predicting the next sentence--but have not been tuned for IR at all.

In [9]:
del knrm # clear out memory from KNRM (useful for GPU)
bert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

Let's see how this non-IR trained model does on CORD10 data

In [10]:
pipeline3 = br % 100 >> pt.text.get_text(dataset, 'abstract') >> bert
pt.Experiment(
    [br, pipeline3],
    topics,
    qrels,
    names=['DPH', 'DPH >> VBERT'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-20 23:55:54,139][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-20 23:55:54,383][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-20 23:56:53,961][onir_pt][DEBUG] [finished] batches: [59.58s] [1250it] [20.98it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,45.319237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> VBERT,0.056413,0.458,0.147048,0.374197,1240.290285,8.0,42.0,0.000006,6.0,37.0,9.462198e-07,5.0,45.0,0.000001,8.0,41.0,4.134415e-09


As we see, although the ERT model is pre-trained for recognizing language, it doesn't do very well at ranking on our benchmark. To get better performance, we'll need to tune for the task of relevance ranking.

We can train the model for ranking (as shown above for KNRM) or we can download a trained model. Here, we will use the [SLEDGE](https://arxiv.org/abs/2010.05987) model, which is a BERT model trained on scientific text and tuned on medical queries.

In [11]:
bert = onir_pt.reranker.from_checkpoint('http://jurgens.people.si.umich.edu/ir/scibert-medmarco.tar.gz', 
                                         text_field='abstract', expected_md5="854966d0b61543ffffa44cea627ab63b")

[2021-11-20 23:56:54,111][onir_pt][INFO] using cached checkpoint: /home/cyuntao/data/onir/model_checkpoints/1f28b1381d23b0062a219452a9258032


Let's run another experiment to see how this new model trained for IR does.

In [12]:
pipeline4 = br % 100 >> pt.text.get_text(dataset, 'abstract') >> bert
pt.Experiment(
    [br, pipeline4],
    topics,
    qrels,
    names=['DPH', 'DPH >> Trained-BERT'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-20 23:57:29,481][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-20 23:57:29,636][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-20 23:58:25,578][onir_pt][DEBUG] [finished] batches: [55.94s] [1250it] [22.35it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,52.679956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> Trained-BERT,0.075710,0.770,0.173079,0.701995,1167.512840,36.0,14.0,0.001278,28.0,11.0,0.000851,36.0,14.0,0.010118,31.0,19.0,0.012156


Training helped a lot! We're able to improve upon the initial ranking from `BatchRetrieve`. However, from looking at `mrt` we can see that this is pretty slow to run--and this was using a GPU! This performance time underscores the trade-off in using large language models at retrieval time: they may perform better, but could be much slower.

# Deep learning at indexing time: doc2query

Instead of using our large language models to rerank, another option is to use them at _indexing time_ to augment our documents. In class, we discussed one such option, doc2query, that augments an inverted index structure by predicting queries that may be used to search for the document, and appending those to the document text.

We can use doc2query using the `pyterrier_doc2query` package, which was loaded at the top.

### Loading a pre-trained model

We'll start by using a version of the doc2query model released by the authors that is trained on the MS MARCO collection.

In [13]:
if not os.path.exists("t5-base.zip"):
  !wget http://jurgens.people.si.umich.edu/ir/t5-base.zip
  !unzip t5-base.zip

We can load the model weights by specifying the checkpoint.

In [14]:
doc2query = pyterrier_doc2query.Doc2Query('model.ckpt-1004000', batch_size=8)

Doc2query using cuda


### Running doc2queries on sample text

Let's see what queries it predicts for the sample document that we've made up:

In [15]:
df = pd.DataFrame([{"docno" : "d1", "text" :"The University of Michigan School of Information (UMSI) delivers innovative, elegant and ethical solutions connecting people, information and technology. The school was one of the first iSchools in the nation and is the premier institution studying and using technology to improve human computer interactions."}])
df.iloc[0].text

'The University of Michigan School of Information (UMSI) delivers innovative, elegant and ethical solutions connecting people, information and technology. The school was one of the first iSchools in the nation and is the premier institution studying and using technology to improve human computer interactions.'

In [16]:
doc2query_df = doc2query(df)
doc2query_df.iloc[0].querygen

'what is umsi university of michigan school of information where is umsi'

Not too bad, though the questions are somewhat generic

### Loading an index of doc2query documents

Let's see how doc2query does on improving the performance in the TREC COVID data. Since indexing with doc2query takes a while (due to needing to run the deep learning models), we've provided an index with the text already added.

If you would like to index the collection with doc2query yourself (or use doc2query for your course project), you can use the following code:

```python
dataset = pt.get_dataset("irds:cord19/trec-covid")
indexer = (
  pyterrier_doc2query.Doc2Query('model.ckpt-1004000', doc_attr='abstract', batch_size=8, append=True) # aply doc2query on abstracts and append
  >> pt.apply.generic(lambda df: df.rename(columns={'abstract': 'text'}) # rename "abstract" column to "text" for indexing
  >> pt.IterDictIndexer("./doc2query_index_path")) # index the expanded documents
indexref = indexer.index(dataset.get_corpus_iter())
```


In [17]:
if not os.path.exists('doc2query_marco_cord19.zip'):
  !wget http://jurgens.people.si.umich.edu/ir/doc2query_marco_cord19.zip
  !unzip doc2query_marco_cord19.zip
doc2query_indexref = pt.IndexRef.of('./doc2query_index_path/data.properties')

Let's look at the results on TREC COVID by first merging the scores with the rankings

In [18]:
dataset = pt.get_dataset('irds:cord19/trec-covid')
pipeline = pt.BatchRetrieve(doc2query_indexref) % 1 >> pt.text.get_text(dataset, 'title')
res = pipeline(dataset.get_topics('title'))
res.merge(dataset.get_qrels(), how='left').head()

/home/cyuntao/miniconda3/envs/650/lib/python3.9/site-packages/pyterrier/datasets.py:430: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,qid,docid,docno,rank,score,query,title,label,iteration
0,1,101299,jwmrgy5d,0,8.427298,coronavirus origin,COVID-19 in the heart and the lungs: could we ...,0.0,5
1,2,182167,g8grcy5j,0,13.922648,coronavirus response to weather changes,The Stirling Protocol – Putting the environmen...,0.0,4
2,3,85678,tl30wlpy,0,7.224180,coronavirus immunity,Receptor-dependent coronavirus infection of de...,NaN,NaN
3,4,145871,l5fxswfz,0,12.773362,how do people die from the coronavirus,Analysis on 54 Mortality Cases of Coronavirus ...,2.0,1.5
4,5,180990,3sepefqa,0,12.995980,animal models of covid 19,Current global vaccine and drug efforts agains...,0.0,4


What kind of queries does doc2query generate for the CORD19 documents?

In [19]:
df = pd.DataFrame(doc for doc in dataset.get_corpus_iter() if doc['docno'] in ('3sepefqa', 'l5fxswfz'))
df = df.rename(columns={'abstract': 'text'})
doc2query_df = doc2query(df)
for querygen, docno, text in zip(doc2query_df['querygen'], doc2query_df['docno'], df['text']):
    print(docno)
    print(querygen)
    print(text)

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

l5fxswfz
what is the population of coronavirus coronavirus death toll coronavirus worldwide
Since the identification of the first case of coronavirus disease 2019 (COVID-19), the global number of confirmed cases as of March 15, 2020, is 156,400, with total death in 5,833 (3.7%) worldwide. Here, we summarize the morality data from February 19 when the first mortality occurred to 0 am, March 10, 2020, in Korea with comparison to other countries. The overall case fatality rate of COVID-19 in Korea was 0.7% as of 0 am, March 10, 2020.
3sepefqa
what is the cure for copid what is the treatment of covid what is a cure for covid 19?
COVID-19 has become one of the biggest health concern, along with huge economic burden. With no clear remedies to treat the disease, doctors are repurposing drugs like chloroquine and remdesivir to treat COVID-19 patients. In parallel, research institutes in collaboration with biotech companies have identified strategies to use viral proteins as vaccine candidates 

## Evaluating the effects of doc2query

Here, we'll change our evaluation setup a bit from what we did before. Rather than compare two models for the same index, we'll instead compare the same model (BM25) with two different ways of indexing (two indices)! Our baseline will be an index of CORD19 without the doc2query additions.

Let's load a copy of the CORD19 index that we used earlier.

In [20]:
indexref = pt.IndexRef.of('./terrier_trec_covid/data.properties')

### Task 2: Write the `Experiment` to compare indices (3 points)
Run an `Experiment` using a `BM25` ranker that compares the indices `indexref` and `doc2query_indexref`. Compare your models using MAP, NDCG, and NDCG@10. Note that our doc2query model was trained on MS MARCO, which isn't the same kind of collection as CORD19, so this performance tells us how well that model can transfer to a new setting.

In [21]:
# TODO
indexref_bm25 = pt.BatchRetrieve(indexref, wmodel="BM25")
doc2query_indexref_bm25 = pt.BatchRetrieve(doc2query_indexref, wmodel="BM25")

pt.Experiment(
    [indexref_bm25, doc2query_indexref_bm25],
    topics,
    qrels, 
    names=["indexref_bm25",  "doc2query_indexref_bm25"],
    eval_metrics=["map", "ndcg", "ndcg_cut_10"])

,name,map,ndcg,ndcg_cut_10
0,indexref_bm25,0.195498,0.411103,0.644463
1,doc2query_indexref_bm25,0.196293,0.412673,0.623652


# Task 3: Train a new model! (25 points)

All of the prior exercises have had you working with either off-the-shelf models (not trained for IR) or models that someone else has trained for you. To give you a sense of how to train a model, your primary task in this notebook is to train a simple `knrm` model, which should be relatively efficient to train on a GPU. 

To keep things simple, we'll use the same setup for CORD19 that we did in Part 2 (30 queries in train, 5 in dev, 15 queries in test) which is still relatively small for training a deep learning model but will get you started on the process. 

Your tasks are the following:
- Load the CORD19 dataset and split it into train, dev, and test
- Create a new `knrm` ranker and a pipeline that uses it
- Run an `Experiment` comparing four models:
  - a default `BatchRetrieve`, filtering to the top 100 results
  - BM25, filtering to the top 100 results
  - a pipeline that feeds the top 100 results of the default `BatchRetrieve` to your `knrm` model
  - a pipeline that feeds the top 100 results of BM25 to your `knrm` model
  
Your `Experiment` should evaluate models using MAP, NDCG, NDCG@10, Precision@10, and Mean Response Time.
  
We expect to see the `Experiment`'s results in the final cell. You are, of course, welcome to try training any of the fancier models to see how they do as well!

In [22]:
# TODO
from sklearn.model_selection import train_test_split

# cord19 has been loaded above
topics = cord19.get_topics(variant='title')

SEED=42
tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)

knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

br = pt.BatchRetrieve(index) % 100

pipeline = (br >> pt.text.get_text(cord19, 'abstract') 
            >> knrm) 

pipeline.fit(
    train_topics,
    cord19.get_qrels(),
    valid_topics,
    cord19.get_qrels())

/home/cyuntao/miniconda3/envs/650/lib/python3.9/site-packages/pyterrier/datasets.py:430: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


[2021-11-20 23:58:55,423][WordvecHashVocab][DEBUG] [starting] reading cached at /home/cyuntao/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2021-11-20 23:58:56,827][WordvecHashVocab][DEBUG] [finished] reading cached at /home/cyuntao/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [1.40s]
[2021-11-20 23:59:03,843][onir_pt][DEBUG] [starting] validation
[2021-11-20 23:59:03,844][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-20 23:59:04,155][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2021-11-20 23:59:04,398][onir_pt][DEBUG] [finished] batches: s] [125it] [516.31it/s]
[2021-11-20 23:59:04,501][onir_pt][DEBUG] [finished] validation s]
[2021-11-20 23:59:04,501][onir_pt][INFO] pre-validation: 0.0095
[2021-11-20 23:59:04,503][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-20 23:59:04,503][onir_pt][DEBUG] [starting] training
[2021-11-20 23:59:04,504][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

...

[2021-11-21 00:00:32,585][onir_pt][DEBUG] [finished] batches: s] [125it] [526.22it/s]
[2021-11-21 00:00:32,683][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 00:00:32,684][onir_pt][INFO] validation it=26 map=0.0098 ndcg=0.0205 P_10=0.0560
[2021-11-21 00:00:32,684][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:00:32,685][onir_pt][DEBUG] [starting] training
[2021-11-21 00:00:32,685][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2021-11-21 00:00:35,574][onir_pt][DEBUG] [finished] train pairs: [2.89s] [1024it] [354.50it/s]
[2021-11-21 00:00:35,575][onir_pt][DEBUG] [finished] training [2.89s]
[2021-11-21 00:00:35,575][onir_pt][INFO] training   it=27 loss=0.2187
[2021-11-21 00:00:35,575][onir_pt][DEBUG] [starting] validation
[2021-11-21 00:00:35,575][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:00:35,576][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2021-11-21 00:00:35,816][onir_pt][DEBUG] [finished] batches: s] [125it] [523.11it/s]
[2021-11-21 00:00:35,914][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 00:00:35,914][onir_pt][INFO] validation it=27 map=0.0098 ndcg=0.0206 P_10=0.0580
[2021-11-21 00:00:35,915][onir_pt][INFO] early stopping; model reverting back to it=7


In [25]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25") % 100
pipeline_bm25 = (bm25 >> pt.text.get_text(cord19, 'abstract') 
            >> knrm)

pipeline_bm25.fit(
    train_topics,
    cord19.get_qrels(),
    valid_topics,
    cord19.get_qrels())

[2021-11-21 00:17:11,497][onir_pt][DEBUG] [starting] validation
[2021-11-21 00:17:11,497][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:17:11,498][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2021-11-21 00:17:11,756][onir_pt][DEBUG] [finished] batches: s] [125it] [483.85it/s]
[2021-11-21 00:17:11,862][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 00:17:11,862][onir_pt][INFO] pre-validation: 0.0114
[2021-11-21 00:17:11,864][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:17:11,864][onir_pt][DEBUG] [starting] training
[2021-11-21 00:17:11,864][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2021-11-21 00:17:14,798][onir_pt][DEBUG] [finished] train pairs: [2.93s] [1024it] [349.07it/s]
[2021-11-21 00:17:14,799][onir_pt][DEBUG] [finished] training [2.94s]
[2021-11-21 00:17:14,799][onir_pt][INFO] training   it=0 loss=0.1666
[2021-11-21 00:17:14,799][onir_pt][DEBUG] [starting] validation
[2021-11-21 00:17:14,800][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:17:14,800][onir_pt][DEBUG] [starting] batches


...

[2021-11-21 00:21:09,875][onir_pt][DEBUG] [finished] batches: s] [125it] [549.59it/s]
[2021-11-21 00:21:09,976][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 00:21:09,977][onir_pt][INFO] validation it=73 map=0.0117 ndcg=0.0229 P_10=0.0700
[2021-11-21 00:21:09,977][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:21:09,977][onir_pt][DEBUG] [starting] training
[2021-11-21 00:21:09,978][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2021-11-21 00:21:12,792][onir_pt][DEBUG] [finished] train pairs: [2.81s] [1024it] [363.94it/s]
[2021-11-21 00:21:12,793][onir_pt][DEBUG] [finished] training [2.82s]
[2021-11-21 00:21:12,793][onir_pt][INFO] training   it=74 loss=0.1684
[2021-11-21 00:21:12,793][onir_pt][DEBUG] [starting] validation
[2021-11-21 00:21:12,793][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:21:12,794][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2021-11-21 00:21:13,019][onir_pt][DEBUG] [finished] batches: s] [125it] [556.24it/s]
[2021-11-21 00:21:13,121][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 00:21:13,122][onir_pt][INFO] validation it=74 map=0.0116 ndcg=0.0228 P_10=0.0700
[2021-11-21 00:21:13,122][onir_pt][INFO] early stopping; model reverting back to it=54


In [26]:
pt.Experiment(
    [br, bm25, pipeline, pipeline_bm25],
    test_topics,
    cord19.get_qrels(),
    names=['BatchRetrieve', 'BM25',  'BatchRetrieve >> knrm', 'BM25 >> knrm'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-21 00:21:15,070][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:21:15,071][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/375 s<?, ?it/s]

[2021-11-21 00:21:15,831][onir_pt][DEBUG] [finished] batches: s] [375it] [493.58it/s]
[2021-11-21 00:21:16,726][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 00:21:16,727][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/375 s<?, ?it/s]

[2021-11-21 00:21:17,326][onir_pt][DEBUG] [finished] batches: s] [375it] [626.22it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,BatchRetrieve,0.057877,0.633333,0.142564,0.528282,36.906543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25,0.054757,0.633333,0.137950,0.537061,36.136694,9.0,6.0,0.369395,4.0,4.0,1.000000,8.0,7.0,0.523367,9.0,5.0,0.829251
2,BatchRetrieve >> knrm,0.059592,0.646667,0.145344,0.565631,88.587252,8.0,7.0,0.582711,6.0,4.0,0.802966,8.0,7.0,0.571835,8.0,6.0,0.482103
3,BM25 >> knrm,0.059984,0.680000,0.144364,0.594605,97.030568,6.0,9.0,0.605938,5.0,5.0,0.491723,6.0,9.0,0.803777,7.0,6.0,0.246699
